In [40]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

In [41]:
cd /Users/scottpowers/Library/CloudStorage/GoogleDrive-Scott.Powers@stonybrook.edu/My Drive/

/Users/scottpowers/My Drive


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [42]:
TCGA_EXPR = "TCGA_EXPR.txt"   # or full path

df_raw = pd.read_csv(TCGA_EXPR, sep="\t", low_memory=False)

print("Raw shape:", df_raw.shape)
print("Column preview:", list(df_raw.columns[:10]))
# ---- 1. Identify gene column automatically ----
gene_col = None
for c in df_raw.columns:
    c_lower = c.lower()
    if ("gene" in c_lower) or ("hybridization" in c_lower) or ("id" == c_lower):
        gene_col = c
        break

if gene_col is None:
    raise ValueError("❗ Could not detect gene column — inspect df_raw.columns manually.")

print("Detected gene column:", gene_col)

# ---- 2. Extract gene symbols safely ----
# If "GENE|1234" → keep "GENE"
df_raw["gene_symbol"] = (
    df_raw[gene_col]
    .astype(str)
    .str.split("|", n=1)
    .str[0]
)

# ---- 3. Identify all numeric sample columns explicitly ----
numeric_cols = []
for col in df_raw.columns:
    if col == gene_col or col == "gene_symbol":
        continue
    # try converting first 5 values to numeric
    sample = pd.to_numeric(df_raw[col].head(5), errors="coerce")
    if sample.notna().sum() >= 3:   # majority numeric
        numeric_cols.append(col)

print(f"Detected {len(numeric_cols)} numeric expression columns.")

# ---- 4. Build a clean gene × sample matrix ----
df_expr = df_raw[["gene_symbol"] + numeric_cols].copy()
df_expr = df_expr.set_index("gene_symbol")

# Convert all values to numeric (safest)
df_expr = df_expr.apply(pd.to_numeric, errors="coerce")

# ---- 5. Collapse duplicate gene symbols safely ----
df_expr = df_expr.groupby(df_expr.index).mean()

# ---- 6. Drop weird gene names only if they exist ----
for bad in ["?", "NA", "na", "", None]:
    if bad in df_expr.index:
        df_expr = df_expr.drop(index=bad, errors="ignore")

print("Final cleaned expression matrix:", df_expr.shape)
display(df_expr.head())


Raw shape: (20532, 184)
Column preview: ['Hybridization REF', 'TCGA-2J-AAB1-01A-11R-A41B-07', 'TCGA-2J-AAB4-01A-12R-A41B-07', 'TCGA-2J-AAB6-01A-11R-A41B-07', 'TCGA-2J-AAB8-01A-12R-A41B-07', 'TCGA-2J-AAB9-01A-11R-A41B-07', 'TCGA-2J-AABA-01A-21R-A41B-07', 'TCGA-2J-AABE-01A-12R-A41B-07', 'TCGA-2J-AABF-01A-31R-A41B-07', 'TCGA-2J-AABH-01A-21R-A41B-07']
Detected gene column: Hybridization REF
Detected 183 numeric expression columns.
Final cleaned expression matrix: (20502, 183)


,TCGA-2J-AAB1-01A-11R-A41B-07,TCGA-2J-AAB4-01A-12R-A41B-07,TCGA-2J-AAB6-01A-11R-A41B-07,TCGA-2J-AAB8-01A-12R-A41B-07,TCGA-2J-AAB9-01A-11R-A41B-07,TCGA-2J-AABA-01A-21R-A41B-07,TCGA-2J-AABE-01A-12R-A41B-07,TCGA-2J-AABF-01A-31R-A41B-07,TCGA-2J-AABH-01A-21R-A41B-07,TCGA-2J-AABI-01A-12R-A41B-07,...,TCGA-XD-AAUH-01A-42R-A41B-07,TCGA-XD-AAUI-01A-42R-A41B-07,TCGA-XD-AAUL-01A-21R-A39D-07,TCGA-XN-A8T3-01A-11R-A36G-07,TCGA-XN-A8T5-01A-12R-A36G-07,TCGA-YB-A89D-11A-11R-A36G-07,TCGA-YB-A89D-01A-12R-A36G-07,TCGA-YH-A8SY-01A-11R-A37L-07,TCGA-YY-A8LH-01A-11R-A36G-07,TCGA-Z5-AAPL-01A-12R-A41B-07
gene_symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,81.9122,56.7551,82.5497,56.9307,105.7878,99.3455,79.4019,92.7368,52.5740,76.3316,...,135.0495,141.4958,101.9751,132.0253,122.8427,140.5795,156.8859,239.3867,65.8274,120.8775
A1CF,25.3659,53.4512,8.1871,33.8425,21.4362,18.7882,3.0831,112.3416,62.2822,0.0000,...,42.6861,85.6860,89.3971,21.0970,9.5352,12.3824,73.9979,2.3364,29.9857,3.9886
A2BP1,0.4878,2.1044,0.0000,0.0000,1.0718,0.0000,0.0000,3.1895,1.3066,0.3628,...,10.9318,2.4765,0.8316,0.0000,4.1716,3.9624,1.0277,0.0000,3.9687,0.0000
A2LD1,180.4976,111.0774,163.1228,185.8143,166.7095,99.2767,134.5645,112.9760,261.8772,56.9993,...,88.4279,158.4200,67.4304,104.5200,89.0226,103.3779,104.0904,87.4533,120.9525,106.7749
A2M,19703.8049,15837.8241,8517.4444,14413.9130,24311.7792,10302.0072,11076.8614,18795.9458,14776.9425,3769.4702,...,23815.5128,20787.3452,22677.6549,18811.5876,20203.6412,31255.4185,23524.5683,8438.5456,4471.2821,10193.6752


In [43]:
# Your module genes (from your message)
IGE_UP = [
"PFN2","SET","LGALS1","PDLIM4","EIF5A","F8A1","BANF1","CTDNEP1","TRAPPC1","AMZ2","PTGES3","ST13","FTH1","PCBP1",
"NAA10","ARPC5L","ETFB","CCDC124","SSR2","PGLS","SELENOH","GSTP1","NDUFB7","AURKAIP1","GCHFR","MYL6","AP2S1",
"S100A13","C9orf16","KRT8","PRKCSH","CST3","PET100","DNAJA2","VPS35","VDAC1","MGST3","PRR13","OCIAD2","FUOM","MIA",
"RPN1","FUCA2","TMED4","ERGIC3","DDOST","NAXE","SNRPD1","SKP1","CNBP","ATP5MC3","SLC25A5","LSM4","NDUFAB1","PCBD1",
"HINT1","ADI1","NENF","MRPL43","VKORC1","EMC4","COA3","NEDD8","CHCHD2","PRDX5","WDR83OS","NDUFA4","GABARAP","PHPT1",
"UFC1","MDH1","ATP5F1E","MRPL41","UBL5","COX5B","ELOB","ATP5ME","UQCRB","SEM1","NDUFB1","COX6C","ATP6V1F","HNRNPA1",
"HIGD2A","POLR2I","METTL26","NDUFB4","OST4","C19orf53","RPL36AL","PPIA","COPS9","COX8A","COX5A","RPL36","RPS15",
"RPS27","RPS11","FTL","SNHG29","NACA","RPS17","RPL23","RPL39","RPS21","RPL9","RPL15","RPL34","RPL27A","RPL36A",
"RPL38","RPS26","RPS14","RPL35","RPL37A","RPL37","RPL12","RPS29"
]
IGE_DOWN = ["STN1","NRG4","IQSEC2","IYD","CHD9","APBB1IP","TMEM238","REX1BD","IFI27","CYBA","METRNL","AKR7A2",
            "SULT1A1","ARHGEF35","AC008397.1","KLK11","DELE1","MISP","FAM234A","CMBL"]

# 

In [52]:
SAT_UP = [
    "ABHD8","AC004870.4","AC005920.1","AC009041.1","AC009309.1","AC011498.1","AC012447.1","AC018521.5",
    "AC018754.1","AC027237.2","AC068338.2","AC072061.1","AC079305.3","AC079807.1","AC087623.2","AC090403.1",
    "AC091271.1","AC092287.1","AC092910.3","AC093323.1","AC099778.1","AC107959.2","AC125611.3","AC144652.1",
    "AC239799.2","AC253572.2","ACBD7","AFMID","AHCY","AL021155.5","AL022069.1","AL031963.3","AL049869.2",
    "AL121574.1","AL133523.1","AL139106.1","AL139246.5","AL355075.4","AL360012.1","AL365436.2","AL592295.5",
    "AL662844.4","AP001160.1","AP002381.2","AP002813.1","ARL4D","ATP2B1-AS1","ATRIP","BAMBI","BHLHE40-AS1",
    "BOLA1","BUD23","C12orf65","C19orf48","C2CD4B","C6orf120","CABYR","CCDC9","CCNE2","CDKN2AIP","CHCHD7",
    "CITED2","CROCC","CSKMT","CTH","DALRD3","DRAIC","DUSP28","EAF2","EIF4A3","FOXA3","FOXL1","GADD45B","GLA",
    "GOT1","GRPEL1","GTF2A1","GTF2B","HEXIM1","HIST1H2AG","HIST1H2AH","HIST1H2AL","HIST1H2BJ","HIST1H2BN",
    "HIST1H3A","HIST1H3J","HIST1H4A","HIST1H4C","HIST1H4E","HIST2H2AC","HIST2H3PS2","HIST3H2A","HIST4H4","HMBS",
    "HSPA2","ID2","IDI1","ING1","KCTD5","KIF9","KLHL11","LAP3","LIFR-AS1","LINC01970","LINC02029","LINC02363",
    "LRG1","LRTOMT","MAFB","MED29","MEPCE","MIR17HG","MORF4L2-AS1","MTHFD2","MYCL","MYOSLID","NANOS1","NPW",
    "NRARP","OAT","OSER1-DT","OSGIN1","PHYH","PICART1","PIEZO1","PIK3R3","PLIN5","PLK2","PMAIP1","PMEL","PNKD",
    "POU3F1","PPP1R3C","PRMT5-AS1","PRR3","PTCH2","PTPN6","RAB26","RALY-AS1","RASL11A","RND1","RNF223","RUVBL2",
    "SAE1","SENP8","SIAH2-AS1","SIRT2","SLC7A5","SNHG12","SNHG5","SNHG8","SREBF2-AS1","SRSF7","STARD5","TBPL1",
    "TCTA","THAP9","TLCD1","TM7SF2","TMEM107","TMEM171","TMEM69","TNFRSF10D","TNK1","TRAM2-AS1","TTC33","UAP1",
    "UBAC2-AS1","UBE2D3-AS1","UBE2S","UGDH","WDR74","Z93241.1","Z99127.4","ZC3H10","ZCWPW1","ZFAS1","ZFX-AS1",
    "ZNF574","ZNF584","ZNF622","ZNF687-AS1","ZNF844","ZNF92","ZSWIM3"
]
SAT_DOWN = [
    "PAX5","AC117386.2","PRSS55","RPS16","GDF7","PAK4","AC022144.1","AC092745.5","AL670729.3",
    "DLEU2L","ELP3","KCNC2","MAP4K1","AL161729.4","SV2C","RGS11","AC005498.1","WFDC5","PSENEN",
    "LINC01956","AC115485.1","CYSLTR2","ASMTL-AS1","AP002001.3","FAM153B"
]
SAT_UP = [g.strip().upper() for g in SAT_UP]
SAT_DOWN = [g.strip().upper() for g in SAT_DOWN]

In [53]:
MPC_UP = [
    "KIAA1211L","CASC4","AHR","YY1AP1","COPA","RELL1","FBXW2","CARMIL1","NUBPL","ZC3H18","AGK","HLCS",
    "TMEM241","URGCP","CADPS2","COBL","ARHGAP42","AC138305.1","FAM135A","SLC41A2","TACC2","MCU","FMN1",
    "LGR4","BAIAP2L1","SGPP2","RGS12","PTPN3","IBTK","SPRY4-AS1","UGT8","PNN","SP140L","PIK3CA","CNOT2",
    "ZFC3H1","UBE2K","STRN3","STAG2","KDM5A","RC3H2","TP53BP1","SLC9A8","ATF7IP","MLLT10","SCAPER","CPSF6",
    "KLC1","HUWE1","TAOK3","ROCK1","MAN2A1","ZNF44","ARGLU1","EHMT1","HTT","SIN3A","RSBN1L","MLLT3","RABGAP1",
    "SCFD1","NEMF","DMXL1","RCOR3","NAA35","WWP1","VPS41","PRRC2B","STX16","ANKRD11","ARHGEF7","SOS2","SLAIN2",
    "LUC7L2","SRPK2","TYW1","HMBOX1","LMBR1","TCF12","MBD5","MON2","COG5","LONP2","RAB3GAP2","LARP4B","NUMB",
    "NF1","NCOA2","ZNF710","CSNK1G1","WDR37","NUTM2B-AS1","ITFG1","UBE3C","MFSD14C","SPG11","PTK2","LPP",
    "ZBTB20","FCHO2","PTPN12","DCUN1D4","KDM7A","SLMAP","KIF13B","MIB1","DIP2C","LRCH1","TNIK","TNKS","SMC5",
    "ANKIB1","RBM33","TNPO3","OSBPL3","RAPGEF2","MED13L","FBXL20","KIAA0232","ITCH","MARCH6","ARID4B","PLEKHA6",
    "FNBP1L","KIAA1217","FARP2","MAGI3","DIAPH2","VPS13A","DGKH","GNAQ","ARHGEF12","MYO1D","CDC42BPA","TTC7A",
    "TRIM44","NSD3","NCOA3","ADNP","RUFY3","RUNX1","TRRAP","PTBP2","ZNF609","CHD7","PARP8","ARIH1","ZHX2",
    "ETV6","CUL1","CDK13","BRAF","MBTD1","AUH","STX17","POLR2J3","KLHDC10","TAF1","TOGARAM1","WRN","ADK",
    "TASOR2","NUDCD3","TBL1X","MOSMO","USP42","CRCP","GALNT11","DDI2","TTC14","UPF2","PDXDC1","ETFDH","SEL1L3",
    "HEATR5A","PSMD5","TBC1D2B","NUP214","GAPVD1","RNF19A","SMURF2","NRIP1","WDFY2","TBK1","LCORL","USP25",
    "RABGEF1","CASD1","RBPJ","AEBP2","MALAT1","MAML2","RBMS1","CRIM1","SNX29","PPFIBP1","SVIL","DDX24","G2E3",
    "RASA1","FAM160A1","RNF24","ESYT2","EPS8","SESTD1","ATP2B4","PELI1","RNF145","B4GALT5","PPP2R2A","NOS1AP",
    "EGFR","CPNE8","ARHGAP21","SMAD3","DAPK1","IGF1R","AFAP1","KLF7","DOCK5","MINDY2","ZXDC","NEDD4","METTL15",
    "FNIP2","NEAT1","CELF1","DANT2","SYT17","TMEM245","ERICH1","ADPGK","LMBRD1","MFSD11","GOLGA2","SCNN1A",
    "XDH","PLEKHA1","PPP2CB","GTF2E2","KCNK1","SPATS2L","RTF1","UACA","VAV2","MDFIC","STAM","CLIP4","KYAT1",
    "MECP2","NUP160","THOC1","LINC-PINT","MCPH1","DISC1","UBA6-AS1","AAK1","NRF1","PHF20","RNF216","PCM1","SAFB2",
    "FAM133B","NR6A1","ATL2","C1orf21","MTUS1","PARD3B","EXT1","ST5","ABI2","INTS6L","TRIM56","PTER","PRR12",
    "RSPH3","TMC5","MECOM","ARHGEF10L","HNF4G","PPM1H","AP005230.1","AC119674.1","ZSCAN18","NOL4L","PDPK1",
    "RSU1","TTC39C","COL27A1","SEC16B","AC005162.3","RASAL1","IL17RA","SPART-AS1","CDC37","MUCL3","TMEM178B",
    "LINC02614","EREG","ELK3","TMTC1","PALM2-AKAP2","RAB31","EMP3"
]


MPC_DOWN = [
    "MRPS15","TALDO1","MDH2","PSMB5","PIK3CD-AS2","DUSP9","POPDC3","AKR1C3","MRPS18A","EIF3H","UQCRC2",
    "IFT22","RAB11B","MBOAT7"
]

MPC_UP = [g.strip().upper() for g in MPC_UP]
MPC_DOWN = [g.strip().upper() for g in MPC_DOWN]


In [54]:
IL2_UP = [
    "ABHD15-AS1","AC007780.1","AC008105.3","AC008964.1","AC016831.1","AC016831.5","AC090772.1","AC108863.2",
    "ADARB1","ALOX5AP","ANKH","ANKRD44","APOBEC3G","ARHGDIB","ARL6IP5","ARSG","ATP10D","BCL11B","BORCS5",
    "CAMK1D","CD4","CD84","CD96","CDC42EP3","CELF2","CERS4","CLEC2D","CNOT6L","CRYBG1","CTSW","DSE","FGD3",
    "FMNL1","FOXN3","FOXO1","FYB1","GFI1","GNAO1","GPRIN3","HOPX","IGF1","IKZF3","IL18R1","INKA2","INSYN2B",
    "IQSEC1","ITPRIPL1","JAK3","KLRC2","KLRC3","KLRF1","KLRG1","LAPTM5","LCP1","LEPROTL1","LINC00513",
    "LINC01237","MAN1A1","MAPRE2","MPHOSPH9","MPP7","MVB12B","MYO5A","NIN","PARP11","PARP15","PCED1B-AS1",
    "PDE3B","PIP4K2A","PLCL1","PLEKHA2","PPP3CC","PRKCH","PRKCQ","PRKD3","PRKX","PTPN22","RAC2","RASGRF2",
    "RFX3","RIPOR2","RNF166","S1PR4","SAMD3","SENP7","SH2B3","SH2D2A","SMARCA2","SNHG26","SPOCK2","SRGN",
    "ST8SIA1","STAT5A","STAT5B","STIM1","STK17A","TMEM200A","TMX4","TRBC2","TRDC","TRIM22","TTN","VAV3",
    "WNT5A-AS1","ZNF101","ZNF471"
]
IL2_DOWN = [
    "RBP7","OVCA2","PLAC4","AC026785.3","LINC02212","LINC00605","AC246817.2","FOXCUT","VGLL2","ZIC4","FOLR3",
    "ECEL1","AC024337.1","C5orf58","AC060814.3","B4GALNT1","UCHL1","VAX1","AL451042.1","COMMD8","IFI30",
    "AL096794.1","RGS10"
]
IL2_UP = [g.strip().upper() for g in IL2_UP]
IL2_DOWN = [g.strip().upper() for g in IL2_DOWN]


In [47]:
clin_cbio = pd.read_csv( "paad_tcga_pan_can_atlas_2018_clinical_data.csv",
    low_memory=False
)


In [55]:
import pandas as pd
import numpy as np

# one-time prep (use the expression matrix you already built)
expr_numeric = df_expr.apply(pd.to_numeric, errors="coerce")

non_numeric_only = expr_numeric.notna().any(axis=1) == False
expr_for_scoring = expr_numeric.loc[~non_numeric_only].copy()

print("Expression matrix for scoring:", expr_for_scoring.shape)


Expression matrix for scoring: (20501, 183)


In [56]:
print(expr_for_scoring.shape)
print(expr_for_scoring.index[:5])     # gene symbols
print(expr_for_scoring.columns[:5])   # TCGA sample IDs
print(expr_for_scoring.dtypes.unique())


(20501, 183)
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M'], dtype='object', name='gene_symbol')
Index(['TCGA-2J-AAB1-01A-11R-A41B-07', 'TCGA-2J-AAB4-01A-12R-A41B-07',
       'TCGA-2J-AAB6-01A-11R-A41B-07', 'TCGA-2J-AAB8-01A-12R-A41B-07',
       'TCGA-2J-AAB9-01A-11R-A41B-07'],
      dtype='object')
[dtype('float64')]


In [57]:
Z = expr_for_scoring.apply(
    lambda x: (x - x.mean()) / (x.std(ddof=0) + 1e-9),
    axis=1
)


In [58]:
import pandas as pd

def check_gene_overlap(expr_index, gene_list, name="GENESET", show_missing=30):
    """
    expr_index: pd.Index or list of gene symbols present in your expression matrix
    gene_list:  list of gene symbols (strings)
    """
    expr_index = pd.Index(expr_index).astype(str)
    genes = pd.Index([str(g) for g in gene_list if pd.notna(g)]).unique()

    present = genes.intersection(expr_index)
    missing = genes.difference(expr_index)

    out = {
        "set": name,
        "total": int(len(genes)),
        "present": int(len(present)),
        "missing": int(len(missing)),
        "pct_present": float(len(present) / max(1, len(genes)) * 100.0),
        "example_missing": list(missing[:show_missing]),
    }
    return out

def overlap_report(expr_for_scoring, gene_sets: dict, show_missing=30):
    """
    gene_sets: dict like {"IGE_UP": IGE_UP, "IGE_DOWN": IGE_DOWN, ...}
    """
    rows = []
    for name, glist in gene_sets.items():
        rows.append(check_gene_overlap(expr_for_scoring.index, glist, name=name, show_missing=show_missing))
    df = pd.DataFrame(rows).sort_values(["pct_present", "total"], ascending=[True, False]).reset_index(drop=True)
    return df

# ---- EXAMPLE USAGE ----
# Replace these with whatever lists you actually have defined in your notebook
gene_sets = {
    "IGE_UP": IGE_UP,
    "IGE_DOWN": IGE_DOWN,
    "SAT_UP": SAT_UP,
    "SAT_DOWN": SAT_DOWN,
    "IL2_UP": IL2_UP,
    "IL2_DOWN": IL2_DOWN,
    "MPC_UP": MPC_UP,
    "MPC_DOWN": MPC_DOWN,
}

rep = overlap_report(expr_for_scoring, gene_sets, show_missing=25)

# Show summary
display(rep[["set", "total", "present", "missing", "pct_present"]])

# Show missing examples for the worst-covered sets
worst = rep.sort_values("pct_present").head(4)
for _, r in worst.iterrows():
    print("\n", r["set"], f"({r['present']}/{r['total']} present; {r['pct_present']:.1f}%):")
    print("Missing examples:", r["example_missing"])


,set,total,present,missing,pct_present
0,IL2_DOWN,23,13,10,56.521739
1,SAT_UP,185,110,75,59.459459
2,SAT_DOWN,25,15,10,60.000000
3,IGE_DOWN,20,13,7,65.000000
4,IL2_UP,106,83,23,78.301887
5,MPC_DOWN,14,12,2,85.714286
6,IGE_UP,118,102,16,86.440678
7,MPC_UP,304,272,32,89.473684



 IL2_DOWN (13/23 present; 56.5%):
Missing examples: ['AC024337.1', 'AC026785.3', 'AC060814.3', 'AC246817.2', 'AL096794.1', 'AL451042.1', 'C5ORF58', 'FOXCUT', 'LINC00605', 'LINC02212']

 SAT_UP (110/185 present; 59.5%):
Missing examples: ['AC004870.4', 'AC005920.1', 'AC009041.1', 'AC009309.1', 'AC011498.1', 'AC012447.1', 'AC018521.5', 'AC018754.1', 'AC027237.2', 'AC068338.2', 'AC072061.1', 'AC079305.3', 'AC079807.1', 'AC087623.2', 'AC090403.1', 'AC091271.1', 'AC092287.1', 'AC092910.3', 'AC093323.1', 'AC099778.1', 'AC107959.2', 'AC125611.3', 'AC144652.1', 'AC239799.2', 'AC253572.2']

 SAT_DOWN (15/25 present; 60.0%):
Missing examples: ['AC005498.1', 'AC022144.1', 'AC092745.5', 'AC115485.1', 'AC117386.2', 'AL161729.4', 'AL670729.3', 'AP002001.3', 'ASMTL-AS1', 'LINC01956']

 IGE_DOWN (13/20 present; 65.0%):
Missing examples: ['AC008397.1', 'DELE1', 'FAM234A', 'MISP', 'REX1BD', 'STN1', 'TMEM238']


In [59]:
import numpy as np
import pandas as pd

def score_all_signed_modules(
    expr,
    modules,
    clin=None,
    patient_id_from_sample=None,
    time_col=None,
    event_col=None,
    covariate_cols=None,
    min_genes=10,
    zscore=True,
    drop_non_numeric_rows=True,
    verbose=True,
):
    """
    Score multiple signed modules (UP - DOWN) from a bulk TCGA-style expression matrix
    and (optionally) merge with clinical survival data into a tidy table.

    Parameters
    ----------
    expr : pd.DataFrame
        Gene x sample expression matrix. Index = gene symbols, columns = sample IDs.
    modules : dict
        dict like:
        {
          "IGE": {"up": IGE_UP, "down": IGE_DOWN},
          "SAT": {"up": SAT_UP, "down": SAT_DOWN},
          ...
        }
        'down' can be [] or None for unsigned sets (then score=mean(Z[up])).
    clin : pd.DataFrame, optional
        Clinical dataframe indexed by patient ID (recommended) OR with a patient ID column.
    patient_id_from_sample : callable, optional
        Function mapping sample barcode -> patient ID.
        Example for TCGA: lambda s: s[:12]
        If None, uses expr.columns as-is.
    time_col, event_col : str, optional
        Column names in clin for survival time and event (1=event, 0=censored).
    covariate_cols : list[str], optional
        Additional clin columns to include (e.g., stage, grade).
    min_genes : int
        Minimum genes required in UP and DOWN lists after intersection.
        For unsigned sets, applies to UP list only.
    zscore : bool
        If True, gene-wise z-score across samples before scoring.
    drop_non_numeric_rows : bool
        If True, drops rows with all NaNs after numeric coercion.
    verbose : bool
        Print coverage summary.

    Returns
    -------
    out : pd.DataFrame
        Tidy table indexed by patient_id (or sample_id if no mapping),
        columns = module scores (+ survival columns if clin provided).
    coverage : pd.DataFrame
        Coverage info per module: used/total UP/DOWN genes.
    Z : pd.DataFrame
        The z-scored matrix used for scoring (returned for reproducibility/debugging).
    """

    if not isinstance(expr, pd.DataFrame):
        raise TypeError("expr must be a pandas DataFrame (genes x samples).")

    # --- coerce to numeric ---
    expr_num = expr.apply(pd.to_numeric, errors="coerce")

    if drop_non_numeric_rows:
        keep = expr_num.notna().any(axis=1)
        expr_num = expr_num.loc[keep].copy()
        if verbose:
            print(f"[score_all] Dropped {(~keep).sum()} all-non-numeric rows; expr_num={expr_num.shape}")

    # --- map sample -> patient id ---
    sample_ids = expr_num.columns.astype(str)
    if patient_id_from_sample is None:
        patient_ids = pd.Index(sample_ids, name="patient_id")
    else:
        patient_ids = pd.Index([patient_id_from_sample(s) for s in sample_ids], name="patient_id")

    # If multiple samples map to same patient, average them (TCGA often has 1, but safe)
    expr_num.columns = patient_ids
    expr_num = expr_num.groupby(level=0, axis=1).mean()

    # --- z-score across patients per gene ---
    if zscore:
        Z = expr_num.sub(expr_num.mean(axis=1), axis=0).div(expr_num.std(axis=1, ddof=0).replace(0, np.nan), axis=0)
        Z = Z.fillna(0.0)
    else:
        Z = expr_num.copy()

    # --- score modules ---
    scores = {}
    cov_rows = []

    for mod_name, spec in modules.items():
        up_genes = spec.get("up", []) or []
        down_genes = spec.get("down", []) or []

        up = pd.Index([str(g) for g in up_genes]).intersection(Z.index)
        dn = pd.Index([str(g) for g in down_genes]).intersection(Z.index)

        cov = {
            "module": mod_name,
            "up_used": int(up.size),
            "up_total": int(len(up_genes)),
            "down_used": int(dn.size),
            "down_total": int(len(down_genes)),
            "pct_up_present": float(100.0 * up.size / max(1, len(up_genes))),
            "pct_down_present": float(100.0 * dn.size / max(1, len(down_genes))) if len(down_genes) else np.nan,
        }
        cov_rows.append(cov)

        # unsigned sets allowed (down empty)
        if len(down_genes) == 0:
            if up.size < min_genes:
                s = pd.Series(np.nan, index=Z.columns, name=f"{mod_name}_score")
            else:
                s = Z.loc[up].mean(axis=0)
                s.name = f"{mod_name}_score"
        else:
            if up.size < min_genes or dn.size < min_genes:
                s = pd.Series(np.nan, index=Z.columns, name=f"{mod_name}_score")
            else:
                s = Z.loc[up].mean(axis=0) - Z.loc[dn].mean(axis=0)
                s.name = f"{mod_name}_score"

        scores[s.name] = s

    score_df = pd.DataFrame(scores)
    coverage = pd.DataFrame(cov_rows).set_index("module")

    if verbose:
        print("\n[score_all] Coverage summary:")
        display(coverage[["up_used","up_total","pct_up_present","down_used","down_total","pct_down_present"]])

    # --- optionally merge with clinical ---
    out = score_df.copy()
    if clin is not None:
        if not isinstance(clin, pd.DataFrame):
            raise TypeError("clin must be a pandas DataFrame.")

        # Ensure clin index is patient_id
        clin2 = clin.copy()
        if clin2.index.name is None or clin2.index.name.lower() not in ("patient_id", "patient", "case_id", "pat_id"):
            # if there's a common patient ID column, user should set index beforehand;
            # we still try a reasonable default:
            for cand in ["PATIENT_ID", "patient_id", "Patient_ID", "case_id", "CASE_ID"]:
                if cand in clin2.columns:
                    clin2 = clin2.set_index(cand)
                    break

        keep_cols = []
        if time_col is not None: keep_cols.append(time_col)
        if event_col is not None: keep_cols.append(event_col)
        if covariate_cols: keep_cols.extend(list(covariate_cols))
        if len(keep_cols) == 0:
            raise ValueError("If clin is provided, set time_col and event_col (and optionally covariate_cols).")

        clin_keep = clin2[keep_cols].copy()
        out = out.join(clin_keep, how="inner")

        if verbose:
            print(f"[score_all] After joining clin: {out.shape[0]} patients, {out.shape[1]} columns")

    return out, coverage, Z


# --------------------------
# EXAMPLE USAGE
# --------------------------
# modules = {
#     "IGE": {"up": IGE_UP, "down": IGE_DOWN},
#     "SAT": {"up": SAT_UP, "down": SAT_DOWN},
#     "IL2": {"up": IL2_UP, "down": IL2_DOWN},
#     "MPC": {"up": MPC_UP, "down": MPC_DOWN},
# }
#
# # expression matrix you built earlier (genes x samples)
# expr = df_expr  # or expr_for_scoring
#
# # clinical table (index should be TCGA-XX-YYYY patient IDs)
# clin = clin_cbio.set_index("PATIENT_ID")  # if not already indexed
#
# surv_df, cov_tbl, Z = score_all_signed_modules(
#     expr=expr,
#     modules=modules,
#     clin=clin,
#     patient_id_from_sample=lambda s: s[:12],  # TCGA patient ID
#     time_col="OS_MONTHS",
#     event_col="OS_STATUS",   # may need recoding to 0/1
#     covariate_cols=["AJCC_PATHOLOGIC_STAGE", "GRADE"],
#     min_genes=10,
#     verbose=True
# )
#
# display(surv_df.head())
# print(surv_df.shape)


In [60]:
import numpy as np
import pandas as pd

def build_survival_table_from_modules(
    expr,                      # gene x sample expression matrix (df_expr or expr_for_scoring)
    modules,                   # dict: {"IGE": {"up": [...], "down":[...]}, ...}
    clin,                      # clinical dataframe with columns shown below
    min_genes=10,
    sample_to_patient=lambda s: str(s)[:12],   # TCGA-XX-YYYY
    patient_col="patientId",
    time_col="OS_MONTHS",
    status_col="OS_STATUS",
    covariate_cols=("AJCC_PATHOLOGIC_TUMOR_STAGE", "GRADE", "AGE", "SEX"),
    verbose=True,
):
    # ---- 1) numeric expression + drop all-non-numeric rows ----
    X = expr.apply(pd.to_numeric, errors="coerce")
    X = X.loc[X.notna().any(axis=1)].copy()

    # ---- 2) map sample -> patient and collapse replicates (if any) ----
    patient_ids = [sample_to_patient(c) for c in X.columns.astype(str)]
    X.columns = pd.Index(patient_ids, name="patientId")
    X = X.groupby(level=0, axis=1).mean()

    # ---- 3) z-score per gene across patients (one time) ----
    mu = X.mean(axis=1)
    sd = X.std(axis=1, ddof=0).replace(0, np.nan)
    Z = X.sub(mu, axis=0).div(sd, axis=0).fillna(0.0)

    # ---- 4) score all modules ----
    scores = {}
    cov_rows = []

    for mod, spec in modules.items():
        up_genes = spec.get("up", []) or []
        dn_genes = spec.get("down", []) or []

        up = pd.Index(up_genes).intersection(Z.index)
        dn = pd.Index(dn_genes).intersection(Z.index)

        cov_rows.append({
            "module": mod,
            "up_used": int(len(up)), "up_total": int(len(up_genes)),
            "dn_used": int(len(dn)), "dn_total": int(len(dn_genes)),
            "pct_up": float(100*len(up)/max(1,len(up_genes))),
            "pct_dn": float(100*len(dn)/max(1,len(dn_genes))) if len(dn_genes) else np.nan,
        })

        if len(dn_genes) == 0:
            # unsigned set
            s = Z.loc[up].mean(axis=0) if len(up) >= min_genes else pd.Series(np.nan, index=Z.columns)
        else:
            s = (Z.loc[up].mean(axis=0) - Z.loc[dn].mean(axis=0)) if (len(up) >= min_genes and len(dn) >= min_genes) else pd.Series(np.nan, index=Z.columns)

        scores[f"{mod}_score"] = s

    score_df = pd.DataFrame(scores)
    coverage = pd.DataFrame(cov_rows).set_index("module")

    # ---- 5) prep clinical + recode event ----
    clin2 = clin.copy()

    # keep one row per patient (if multiple samples, take first; usually 1:1)
    clin2 = clin2.drop_duplicates(subset=[patient_col]).set_index(patient_col)

    # OS event: treat anything starting with "1" as event
    os_event = clin2[status_col].astype(str).str.strip().str.startswith("1").astype(int)
    clin_keep = pd.DataFrame({
        "OS_time_months": pd.to_numeric(clin2[time_col], errors="coerce"),
        "OS_event": os_event,
    }, index=clin2.index)

    # add covariates if present
    covariate_cols = list(covariate_cols) if covariate_cols else []
    present_covs = [c for c in covariate_cols if c in clin2.columns]
    if present_covs:
        clin_keep = clin_keep.join(clin2[present_covs])

    # ---- 6) merge scores + clinical ----
    surv_df = score_df.join(clin_keep, how="inner").dropna(subset=["OS_time_months", "OS_event"])

    if verbose:
        print(f"[OK] expr genes x patients: {Z.shape}")
        print(f"[OK] survival table patients: {surv_df.shape[0]}  columns: {surv_df.shape[1]}")
        print("\nCoverage:")
        display(coverage[["up_used","up_total","pct_up","dn_used","dn_total","pct_dn"]].sort_values("pct_up"))

    return surv_df, coverage, score_df


In [61]:
modules = {
    "IGE": {"up": IGE_UP, "down": IGE_DOWN},
    "SAT": {"up": SAT_UP, "down": SAT_DOWN},
    "IL2": {"up": IL2_UP, "down": IL2_DOWN},
    "MPC": {"up": MPC_UP, "down": MPC_DOWN},
}

# clin_cbio is your dataframe loaded from paad_tcga_pan_can_atlas_2018_clinical_data.csv
surv_df, cov_tbl, score_df = build_survival_table_from_modules(
    expr=df_expr,        # or expr_for_scoring
    modules=modules,
    clin=clin_cbio,
    covariate_cols=("AJCC_PATHOLOGIC_TUMOR_STAGE", "GRADE", "AGE", "SEX"),
    min_genes=10,
    verbose=True
)

display(surv_df.head())


[OK] expr genes x patients: (20501, 178)
[OK] survival table patients: 177  columns: 10

Coverage:


/var/folders/mh/r2nrb73509z0t_wjbx9r5fg00000gn/T/ipykernel_50678/3633517700.py:23: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  X = X.groupby(level=0, axis=1).mean()


,up_used,up_total,pct_up,dn_used,dn_total,pct_dn
module,,,,,,
SAT,110,185,59.459459,15,25,60.000000
IL2,83,106,78.301887,13,23,56.521739
IGE,102,118,86.440678,13,20,65.000000
MPC,272,304,89.473684,12,14,85.714286


,IGE_score,SAT_score,IL2_score,MPC_score,OS_time_months,OS_event,AJCC_PATHOLOGIC_TUMOR_STAGE,GRADE,AGE,SEX
patientId,,,,,,,,,,
TCGA-2J-AAB1,-0.509804,-0.247533,-0.060611,-0.094102,2.169839,1,STAGE IIB,G3,65,Male
TCGA-2J-AAB4,-0.217649,-0.414703,0.079749,0.180121,23.966861,0,STAGE IIB,G2,48,Male
TCGA-2J-AAB6,0.617456,0.081355,-0.372486,-0.731198,9.632771,1,STAGE IIA,G2,75,Male
TCGA-2J-AAB8,0.046984,0.112810,-0.202396,0.042995,2.630108,0,STAGE IIB,G3,71,Male
TCGA-2J-AAB9,0.168059,0.161719,-0.010048,-0.047520,20.613473,1,STAGE IIB,G1,70,Female


In [62]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

# ----------------------------
# Settings you can tweak
# ----------------------------
PENALIZER = 0.1          # ridge; try 0.01 if you want weaker, 0.2 if still unstable
MIN_COUNT_PER_LEVEL = 8  # levels with < this count will be collapsed to "Other" (or dropped for GX)
DROP_GRADE_GX = True
COLLAPSE_STAGE = True    # collapse to I / II / III-IV
COLLAPSE_GRADE = True    # collapse to G1-2 / G3-4

# ----------------------------
# 0) Sanity checks
# ----------------------------
if "surv_df" not in globals():
    raise NameError("Missing required variable: surv_df")

required_cols = ["OS_time_months","OS_event","SAT_score","MPC_score","IL2_score","IGE_score"]
missing = [c for c in required_cols if c not in surv_df.columns]
if missing:
    raise KeyError(f"surv_df missing: {missing}")

STAGE_COL = "AJCC_PATHOLOGIC_TUMOR_STAGE"
GRADE_COL = "GRADE"

use_covars = [c for c in [STAGE_COL, GRADE_COL] if c in surv_df.columns]
print("Using covariates:", use_covars if use_covars else "(none)")

# ----------------------------
# 1) Build modeling DF
# ----------------------------
df_mod = surv_df[required_cols + use_covars].copy()
df_mod = df_mod.dropna()
print("[COX] N after dropna:", df_mod.shape[0])

# ----------------------------
# 2) Z-score modules (HR per 1 SD)
# ----------------------------
for col in ["SAT_score","MPC_score","IL2_score","IGE_score"]:
    mu = df_mod[col].mean()
    sd = df_mod[col].std(ddof=0)
    df_mod[col.replace("_score","_Z")] = 0.0 if (sd == 0 or np.isnan(sd)) else (df_mod[col] - mu) / sd

df_model = df_mod[["OS_time_months","OS_event","SAT_Z","MPC_Z","IL2_Z","IGE_Z"]].copy()

# ----------------------------
# 3) Clean / collapse covariates to prevent singularity
# ----------------------------
def collapse_rare_levels(s: pd.Series, min_count=8, drop_level=None):
    s = s.astype(str)
    if drop_level is not None:
        s = s[~s.eq(drop_level)]
    counts = s.value_counts(dropna=False)
    rare = counts[counts < min_count].index.tolist()
    if len(rare) > 0:
        s = s.replace({lvl: "Other" for lvl in rare})
    return s

def collapse_stage_to_bins(stage_series: pd.Series):
    # expects strings like "STAGE IIB", "STAGE IB", etc.
    s = stage_series.astype(str).str.upper()
    def map_stage(x):
        if "IV" in x:
            return "IV"
        if "III" in x:
            return "III"
        if "II" in x:
            return "II"
        if "I" in x:
            return "I"
        return "UNK"
    return s.map(map_stage).replace({"III": "III-IV", "IV": "III-IV"})

def collapse_grade_to_bins(grade_series: pd.Series):
    s = grade_series.astype(str).str.upper()
    def map_grade(x):
        if x in ["G1","1"]:
            return "G1-2"
        if x in ["G2","2"]:
            return "G1-2"
        if x in ["G3","3"]:
            return "G3-4"
        if x in ["G4","4"]:
            return "G3-4"
        if x in ["GX","X","NA","NAN","NONE"]:
            return "GX"
        return "UNK"
    return s.map(map_grade)

# Stage
if STAGE_COL in df_mod.columns:
    st = df_mod[STAGE_COL].copy()
    st = collapse_stage_to_bins(st) if COLLAPSE_STAGE else st.astype(str)
    st = collapse_rare_levels(st, min_count=MIN_COUNT_PER_LEVEL)  # collapse rare stage bins if needed
    df_model = df_model.loc[st.index]  # align
    stage_dum = pd.get_dummies(st.astype("category"), prefix="STAGE", drop_first=True)
    print("Stage levels used:", list(st.astype("category").cat.categories))
    df_model = pd.concat([df_model, stage_dum], axis=1)

# Grade
if GRADE_COL in df_mod.columns:
    gr = df_mod[GRADE_COL].copy()
    gr = collapse_grade_to_bins(gr) if COLLAPSE_GRADE else gr.astype(str).str.upper()
    if DROP_GRADE_GX:
        keep_idx = gr.ne("GX")
        df_model = df_model.loc[keep_idx.index[keep_idx]]
        gr = gr.loc[keep_idx]
        print("Dropped GRADE=GX; remaining N:", df_model.shape[0])

    gr = collapse_rare_levels(gr, min_count=MIN_COUNT_PER_LEVEL)
    grade_dum = pd.get_dummies(gr.astype("category"), prefix="GRADE", drop_first=True)
    print("Grade levels used:", list(gr.astype("category").cat.categories))
    df_model = pd.concat([df_model, grade_dum], axis=1)

df_model = df_model.dropna()
print("[COX] Final N:", df_model.shape[0])
print("[COX] Model columns:", df_model.columns.tolist())

# ----------------------------
# 4) Fit Cox with ridge penalization (stabilizes collinearity)
# ----------------------------
cph = CoxPHFitter(penalizer=PENALIZER)
cph.fit(df_model, duration_col="OS_time_months", event_col="OS_event")

# ----------------------------
# 5) Output tidy table
# ----------------------------
out = cph.summary.copy()
out["HR"] = np.exp(out["coef"])
out["CI95_low"]  = np.exp(out["coef lower 95%"])
out["CI95_high"] = np.exp(out["coef upper 95%"])
out = out[["coef","HR","CI95_low","CI95_high","p"]].sort_values("p")

print("\n[COX MULTIVARIABLE RESULTS]")
display(out)

out_path = "SuppTable_multivariable_cox_modules_plus_stage_grade_stable.csv"
out.to_csv(out_path, index=True)
print(f"\nSaved: {out_path}")


Using covariates: ['AJCC_PATHOLOGIC_TUMOR_STAGE', 'GRADE']
[COX] N after dropna: 175
Stage levels used: ['I', 'II', 'III-IV']
Dropped GRADE=GX; remaining N: 173
Grade levels used: ['G1-2', 'G3-4']
[COX] Final N: 173
[COX] Model columns: ['OS_time_months', 'OS_event', 'SAT_Z', 'MPC_Z', 'IL2_Z', 'IGE_Z', 'STAGE_II', 'STAGE_III-IV', 'GRADE_G3-4']

[COX MULTIVARIABLE RESULTS]


,coef,HR,CI95_low,CI95_high,p
covariate,,,,,
STAGE_II,0.719198,2.052786,1.090967,3.862565,0.025753
GRADE_G3-4,0.289413,1.335643,0.886937,2.011353,0.165882
IGE_Z,0.112995,1.119626,0.859202,1.458986,0.402862
SAT_Z,0.092241,1.096629,0.875360,1.373828,0.422426
IL2_Z,0.097530,1.102445,0.843764,1.440432,0.474713
STAGE_III-IV,0.293626,1.341282,0.424551,4.237505,0.616879
MPC_Z,-0.000754,0.999247,0.760688,1.312620,0.995680



Saved: SuppTable_multivariable_cox_modules_plus_stage_grade_stable.csv


In [63]:
import numpy as np
import pandas as pd

# Requires: surv_df with SAT_score, MPC_score, IL2_score, IGE_score (and optionally an existing combined column)
# and OS_time_months / OS_event if you do the optional KM part below.

req = ["SAT_score","MPC_score","IL2_score","IGE_score"]
missing = [c for c in req if c not in surv_df.columns]
if missing:
    raise KeyError(f"surv_df missing required columns: {missing}")

# --- 1) Z-score individual modules (same as your workflow) ---
df = surv_df.copy()
for col in ["SAT_score","MPC_score","IL2_score","IGE_score"]:
    mu = df[col].mean()
    sd = df[col].std(ddof=0)
    df[col.replace("_score","_Z")] = 0.0 if (sd == 0 or np.isnan(sd)) else (df[col] - mu) / sd

# --- 2) Define combined scores both ways ---
df["Combined3_Z"] = df[["SAT_Z","MPC_Z","IL2_Z"]].mean(axis=1)
df["Combined4_Z"] = df[["SAT_Z","MPC_Z","IL2_Z","IGE_Z"]].mean(axis=1)

print("Combined3_Z / Combined4_Z computed.")
print(df[["Combined3_Z","Combined4_Z"]].describe())

# --- 3) If you already have an 'old combined' score, compare it ---
candidate_old_cols = [
    "Combined_Z", "combined_z", "combined_score", "Combined_score",
    "combined_module", "Combined_module", "combined_module_score", "Combined_module_score"
]
old_col = next((c for c in candidate_old_cols if c in df.columns), None)

def compare(a: pd.Series, b: pd.Series, name_a="A", name_b="B"):
    both = pd.concat([a, b], axis=1).dropna()
    if both.shape[0] == 0:
        return {"n": 0}
    x = both.iloc[:,0].astype(float)
    y = both.iloc[:,1].astype(float)
    diff = (x - y)
    return {
        "n": int(both.shape[0]),
        "pearson_r": float(x.corr(y, method="pearson")),
        "spearman_r": float(x.corr(y, method="spearman")),
        "max_abs_diff": float(np.max(np.abs(diff))),
        "mean_abs_diff": float(np.mean(np.abs(diff))),
    }

if old_col is not None:
    print(f"\nFound existing combined column: {old_col}")
    cmp3 = compare(df[old_col], df["Combined3_Z"], name_a=old_col, name_b="Combined3_Z")
    cmp4 = compare(df[old_col], df["Combined4_Z"], name_a=old_col, name_b="Combined4_Z")
    print("Compare old vs Combined3_Z:", cmp3)
    print("Compare old vs Combined4_Z:", cmp4)

    # Simple decision
    if cmp3.get("pearson_r", 0) > cmp4.get("pearson_r", 0) and cmp3.get("max_abs_diff", 1e9) < cmp4.get("max_abs_diff", 1e9):
        print("\n>>> Old combined matches Combined3_Z better (likely 3-module).")
    elif cmp4.get("pearson_r", 0) > cmp3.get("pearson_r", 0) and cmp4.get("max_abs_diff", 1e9) < cmp3.get("max_abs_diff", 1e9):
        print("\n>>> Old combined matches Combined4_Z better (likely 4-module).")
    else:
        print("\n>>> Not a clean match (old combined may have been computed differently: different scaling, weights, or missingness handling).")
else:
    print("\nNo existing combined column found in surv_df.")
    print("If you computed it earlier in code (a Series), paste its variable name or add it to surv_df to compare.")

# --- 4) OPTIONAL: reproduce the KM optimal cutpoint for both combined definitions ---
# This is only useful if you want to see which one recreates your reported cut/p.
DO_KM = True

if DO_KM:
    km_req = ["OS_time_months","OS_event"]
    miss_km = [c for c in km_req if c not in df.columns]
    if miss_km:
        print("\n[KM] Skipping KM check (missing):", miss_km)
    else:
        from lifelines import KaplanMeierFitter
        from lifelines.statistics import logrank_test

        def best_cutpoint(series, time, event, grid_q=(0.2, 0.8), n_grid=200):
            """Brute-force cutpoint over quantile range; returns best cut by logrank p."""
            tmp = pd.concat([series, time, event], axis=1).dropna()
            s = tmp.iloc[:,0].astype(float)
            t = tmp[time.name]
            e = tmp[event.name]
            lo = s.quantile(grid_q[0])
            hi = s.quantile(grid_q[1])
            cuts = np.linspace(lo, hi, n_grid)
            best = {"cut": None, "p": 1.0}
            for c in cuts:
                g_hi = s >= c
                if g_hi.sum() < 10 or (~g_hi).sum() < 10:
                    continue
                res = logrank_test(t[g_hi], t[~g_hi], e[g_hi], e[~g_hi])
                p = float(res.p_value)
                if p < best["p"]:
                    best = {"cut": float(c), "p": p, "n_hi": int(g_hi.sum()), "n_lo": int((~g_hi).sum())}
            return best

        time = df["OS_time_months"]
        event = df["OS_event"]

        best3 = best_cutpoint(df["Combined3_Z"], time, event)
        best4 = best_cutpoint(df["Combined4_Z"], time, event)

        print("\n[KM] Best cut Combined3_Z:", best3)
        print("[KM] Best cut Combined4_Z:", best4)
        print("\n>>> Whichever is closer to your Fig 7 (cut≈0.80, p≈0.036) is likely what you actually used.")


Combined3_Z / Combined4_Z computed.
       Combined3_Z   Combined4_Z
count   177.000000  1.770000e+02
mean      0.000000  1.254489e-17
std       0.499444  3.000018e-01
min      -2.102480 -9.334552e-01
25%      -0.244612 -2.056606e-01
50%       0.041842  2.622053e-02
75%       0.321267  2.097176e-01
max       1.043712  8.637098e-01

No existing combined column found in surv_df.
If you computed it earlier in code (a Series), paste its variable name or add it to surv_df to compare.

[KM] Best cut Combined3_Z: {'cut': 0.24159896245274753, 'p': 0.015136850666223483, 'n_hi': 55, 'n_lo': 122}
[KM] Best cut Combined4_Z: {'cut': 0.25864264571153706, 'p': 0.06043238860297067, 'n_hi': 36, 'n_lo': 141}

>>> Whichever is closer to your Fig 7 (cut≈0.80, p≈0.036) is likely what you actually used.


In [64]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

PENALIZER = 0.1  # ridge stabilizer

# ----------------------------
# 0) Sanity checks
# ----------------------------
if "surv_df" not in globals():
    raise NameError("surv_df not found")

for c in ["OS_time_months", "OS_event"]:
    if c not in surv_df.columns:
        raise KeyError(f"surv_df is missing required column: {c}")

# ----------------------------
# 1) Pick the right module columns (scores, not already-z unless that's all you have)
# ----------------------------
def first_present(cols):
    for c in cols:
        if c in surv_df.columns:
            return c
    return None

SAT_COL = first_present(["SAT_score", "SAT", "SAT_Z"])
MPC_COL = first_present(["MPC_score", "MPC", "MPC_Z"])
IL2_COL = first_present(["IL2_score", "IL2", "IL2_Z"])

if SAT_COL is None or MPC_COL is None or IL2_COL is None:
    raise KeyError(
        "Could not find SAT/MPC/IL2 columns in surv_df. "
        f"Found SAT={SAT_COL}, MPC={MPC_COL}, IL2={IL2_COL}. "
        "Print surv_df.columns and adjust names."
    )

print("Using columns:", {"SAT": SAT_COL, "MPC": MPC_COL, "IL2": IL2_COL})

# ----------------------------
# 2) Create Combined3_Z INSIDE surv_df
#    (average of standardized SAT/MPC/IL2)
# ----------------------------
tmp = surv_df[[SAT_COL, MPC_COL, IL2_COL]].apply(pd.to_numeric, errors="coerce").copy()

# if you already passed *_Z columns, they are already standardized; otherwise z-score them
def zscore(s):
    s = s.astype(float)
    mu = s.mean()
    sd = s.std(ddof=0)
    return (s - mu) / sd if (sd and not np.isnan(sd)) else s*0.0

Z = tmp.copy()
# Only zscore if the column name doesn't already look like Z
for col in Z.columns:
    if not col.upper().endswith("_Z"):
        Z[col] = zscore(Z[col])

surv_df = surv_df.copy()  # avoid side-effects on original
surv_df["Combined3_Z"] = Z.mean(axis=1)

print("Combined3_Z added to surv_df.")
print(surv_df["Combined3_Z"].describe())

# ----------------------------
# 3) Build Cox modeling dataframe
# ----------------------------
STAGE_COL = "AJCC_PATHOLOGIC_TUMOR_STAGE"
GRADE_COL = "GRADE"

req_cols = ["OS_time_months", "OS_event", "Combined3_Z", STAGE_COL, GRADE_COL]
missing = [c for c in req_cols if c not in surv_df.columns]
if missing:
    raise KeyError(f"surv_df missing: {missing}")

df = surv_df[req_cols].dropna().copy()
print("[COX] N after initial dropna:", df.shape[0])

# ----------------------------
# 4) Collapse stage / grade to avoid singular matrices
# ----------------------------
def collapse_stage(x):
    x = str(x).upper()
    if "IV" in x or "III" in x:
        return "III-IV"
    if "II" in x:
        return "II"
    if "I" in x:
        return "I"
    return np.nan

def collapse_grade(x):
    x = str(x).upper()
    if x in ["G1","G2","1","2"]:
        return "G1-2"
    if x in ["G3","G4","3","4"]:
        return "G3-4"
    return np.nan  # drop GX/UNK/etc to keep stable

df["STAGE_BIN"] = df[STAGE_COL].map(collapse_stage)
df["GRADE_BIN"] = df[GRADE_COL].map(collapse_grade)
df = df.dropna(subset=["STAGE_BIN", "GRADE_BIN"]).copy()

print("Stage levels used:", sorted(df["STAGE_BIN"].unique().tolist()))
print("Grade levels used:", sorted(df["GRADE_BIN"].unique().tolist()))
print("[COX] N after collapsing stage/grade:", df.shape[0])

# ----------------------------
# 5) Model matrix
# ----------------------------
X = pd.concat(
    [
        df[["OS_time_months", "OS_event", "Combined3_Z"]],
        pd.get_dummies(df["STAGE_BIN"], prefix="STAGE", drop_first=True),
        pd.get_dummies(df["GRADE_BIN"], prefix="GRADE", drop_first=True),
    ],
    axis=1,
).dropna()

print("[COX] Final N:", X.shape[0])
print("[COX] Columns:", list(X.columns))

# ----------------------------
# 6) Fit Cox
# ----------------------------
cph = CoxPHFitter(penalizer=PENALIZER)
cph.fit(X, duration_col="OS_time_months", event_col="OS_event")

out = cph.summary.copy()
out["HR"] = np.exp(out["coef"])
out["CI95_low"]  = np.exp(out["coef lower 95%"])
out["CI95_high"] = np.exp(out["coef upper 95%"])
out = out[["coef","HR","CI95_low","CI95_high","p"]].sort_values("p")

print("\n[COX MULTIVARIABLE RESULTS: Combined3_Z + Stage + Grade]")
display(out)

out_path = "SuppTable_multivariable_cox_Combined3Z_plus_stage_grade.csv"
out.to_csv(out_path, index=True)
print(f"\nSaved: {out_path}")


Using columns: {'SAT': 'SAT_score', 'MPC': 'MPC_score', 'IL2': 'IL2_score'}
Combined3_Z added to surv_df.
count    177.000000
mean       0.000000
std        0.499444
min       -2.102480
25%       -0.244612
50%        0.041842
75%        0.321267
max        1.043712
Name: Combined3_Z, dtype: float64
[COX] N after initial dropna: 175
Stage levels used: ['I', 'II', 'III-IV']
Grade levels used: ['G1-2', 'G3-4']
[COX] N after collapsing stage/grade: 173
[COX] Final N: 173
[COX] Columns: ['OS_time_months', 'OS_event', 'Combined3_Z', 'STAGE_II', 'STAGE_III-IV', 'GRADE_G3-4']

[COX MULTIVARIABLE RESULTS: Combined3_Z + Stage + Grade]


,coef,HR,CI95_low,CI95_high,p
covariate,,,,,
STAGE_II,0.673459,1.961009,1.045817,3.677085,0.035761
GRADE_G3-4,0.270701,1.310884,0.872304,1.969973,0.192718
STAGE_III-IV,0.269443,1.309234,0.415892,4.121491,0.645152
Combined3_Z,0.041418,1.042287,0.692625,1.568473,0.842552



Saved: SuppTable_multivariable_cox_Combined3Z_plus_stage_grade.csv


In [65]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

# ----------------------------
# Required columns
# ----------------------------
base_cols = ["OS_time_months", "OS_event",
             "SAT_score", "MPC_score", "IL2_score"]

df = surv_df[base_cols].dropna().copy()
print("N:", df.shape[0])

# ----------------------------
# Z-score helpers
# ----------------------------
def zscore(s):
    return (s - s.mean()) / s.std(ddof=0)

df["SAT_Z"] = zscore(df["SAT_score"])
df["MPC_Z"] = zscore(df["MPC_score"])
df["IL2_Z"] = zscore(df["IL2_score"])

df["SAT_MPC_Z"] = zscore(df[["SAT_Z","MPC_Z"]].mean(axis=1))
df["SAT_IL2_Z"] = zscore(df[["SAT_Z","IL2_Z"]].mean(axis=1))
df["MPC_IL2_Z"] = zscore(df[["MPC_Z","IL2_Z"]].mean(axis=1))
df["Combined3_Z"] = zscore(df[["SAT_Z","MPC_Z","IL2_Z"]].mean(axis=1))

predictors = [
    "SAT_Z",
    "MPC_Z",
    "IL2_Z",
    "SAT_MPC_Z",
    "SAT_IL2_Z",
    "MPC_IL2_Z",
    "Combined3_Z",
]

rows = []

for pred in predictors:
    cph = CoxPHFitter()
    cph.fit(
        df[["OS_time_months","OS_event",pred]],
        duration_col="OS_time_months",
        event_col="OS_event"
    )
    s = cph.summary.loc[pred]
    rows.append({
        "predictor": pred,
        "HR_per_SD": np.exp(s["coef"]),
        "p": s["p"],
        "log_likelihood": cph.log_likelihood_,
        "AIC": -2*cph.log_likelihood_ + 2*1
    })

res = pd.DataFrame(rows).sort_values("AIC")
display(res)


N: 177


,predictor,HR_per_SD,p,log_likelihood,AIC
4,SAT_IL2_Z,1.135947,0.241660,-407.151009,816.302018
3,SAT_MPC_Z,1.130057,0.266071,-407.199979,816.399958
6,Combined3_Z,1.115305,0.303479,-407.297134,816.594268
0,SAT_Z,1.069380,0.531646,-407.649308,817.298616
1,MPC_Z,1.062191,0.551421,-407.666071,817.332142
5,MPC_IL2_Z,1.054337,0.603923,-407.711608,817.423217
2,IL2_Z,1.037393,0.727176,-407.787430,817.574859


In [66]:
surv_df[["SAT_score","MPC_score","IL2_score"]].corr()


,SAT_score,MPC_score,IL2_score
SAT_score,1.000000,-0.379819,-0.645503
MPC_score,-0.379819,1.000000,0.641481
IL2_score,-0.645503,0.641481,1.000000


In [67]:
surv_df = clin_df.copy()

surv_df = surv_df.join(SAT_scores.rename("SAT_score"))
surv_df = surv_df.join(MPC_scores.rename("MPC_score"))
surv_df = surv_df.join(IL2_scores.rename("IL2_score"))
surv_df = surv_df.join(IGE_scores.rename("IGE_score"))


NameError: name 'clin_df' is not defined